1. Solution using glpk and cbc solvers

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 5.4 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
model_lab3_Ex1 = ConcreteModel()

In [5]:
N = 3 #number of decision variables 
M = 3 #number of constraints without considering the bound constraints  
lb=np.array([2,1,1]) # array for lower bounds
ub=np.array([np.inf, np.inf, np.inf]) #array for upper bounds
obj_coef_c = np.array([2, -2, 2]) #array to store coefficients of decision variables in objective function
constr_coef_A=np.array([[2, -1, 2],[1,-4,2],[-2,2,1]]) #matrix to store coefficients of decision variables in the constraints
constr_rhs_b=np.array([6,-15,-8]) #RHS values of the constraints
row_indices=np.arange(M)
col_indices = np.arange(N)

In [6]:
model_lab3_Ex1.x=Var(col_indices)
model_lab3_Ex1.constraints = ConstraintList()

In [7]:
for i in col_indices:
  model_lab3_Ex1.x[i].setlb(lb[i])
  model_lab3_Ex1.x[i].setub(ub[i])


In [8]:
for i in row_indices:
  model_lab3_Ex1.constraints.add(sum(constr_coef_A[i][j]*model_lab3_Ex1.x[j] for j in col_indices) <= constr_rhs_b[i])

In [9]:
model_lab3_Ex1.objective = Objective(expr =summation(obj_coef_c,model_lab3_Ex1.x), sense=minimize)
model_lab3_Ex1.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

We first solve using the glpk solver

In [10]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [11]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
#.solve(model1).write()
result = opt.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.017358064651489258

Solver status: ok
Solver termination condition: other


Cbc solver

In [12]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155879 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [13]:
opt_cbc = SolverFactory('cbc')

In [14]:
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [15]:
result = opt_cbc.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.050652503967285156

Solver status: warning
Solver termination condition: infeasible


1. The messages shown by the glpk and cbc solvers are as follows:  
  
glpk Solver:  
Solver status: ok  
Solver termination condition: other  
  
cbc Solver:  
Solver status: warning  
Solver termination condition: infeasible  
  
The messages coming from the glpk solver are comparatively less meaningful as compared to those coming from the cbc solver.  
The solver termination condition from the glpk solver gives the message 'other', which means that we do not get an optimal solution for the modeled problem. However, the glpk solver is incapable of informing us about the reason behind the absence of an optimal solution.
Cbc solver on the other hand, clearly states that under the provided set of constraints, an optimal solution cannot be obtained, hence, it is infeasible. The model could be infeasible because of the constraints, due to which we may not get any feasible region at all where we may look for possible solutions.  
The solver status message in cbc solver displays the message 'warning' as we are unable to find a solution for the model, whereas the solver status message in glpk solver is unable to convey this message to the user.

2 a. The required changes are done below:

In [16]:
model_lab3_Ex1.objective.set_sense(maximize)

In [17]:
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [18]:
result = opt_cbc.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.016866445541381836

Solver status: warning
Solver termination condition: infeasible


2(a). The solver status and solver termination condition messages obtained are:  
  
Solver status: warning  
Solver termination condition: infeasible  
  
The solver status message reads 'warning' as we are unable to find a solution to the modeled problem.  
The solver termination condition message reads 'infeasible' because given the set of constraints, we are unable to find any solution that satisfies all the stated constraints.

2 b. The required changes are done below:

In [19]:
model_lab3_Ex1.objective.set_sense(minimize)

In [20]:
model_lab3_Ex1.x[1].setub(5)
model_lab3_Ex1.x[2].setub(10)
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :     5 : False :  True :  Reals
          2 :     1 :  None :    10 : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [21]:
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :     5 : False :  True :  Reals
          2 :     1 :  None :    10 : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [22]:
result = opt_cbc.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.020258426666259766

Solver status: warning
Solver termination condition: infeasible


2(b). The solver status and solver termination condition messages obtained are:  
  
Solver status: warning  
Solver termination condition: infeasible  
  
The solver status message reads 'warning' as we are unable to find a solution to the modeled problem.  
The solver termination condition message reads 'infeasible' because given the set of constraints, we are unable to find any solution that satisfies all the stated constraints.

2 c. The required changes are done below:

In [23]:
model_lab3_Ex1.x[1].setub(np.inf)
model_lab3_Ex1.x[2].setub(np.inf)

In [24]:
new_constr_coef = [0,-1,1]
new_constr_rhs = 16
model_lab3_Ex1.constraints.add(sum(new_constr_coef[j]*model_lab3_Ex1.x[j] for j in col_indices) <= new_constr_rhs)
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :  

In [25]:
result = opt_cbc.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: 11.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 0
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01657724380493164

Solver status: warning
Solver termination condition: infeasible


2(c). The solver status and solver termination condition messages obtained are:  
  
Solver status: warning  
Solver termination condition: infeasible  
  
The solver status message reads 'warning' as we are unable to find a solution to the modeled problem.  
The solver termination condition message reads 'infeasible' because given the set of constraints, we are unable to find any solution that satisfies all the stated constraints.

2 d. The required changes are done below:

In [26]:
#deactivate the old objective
model_lab3_Ex1.objective.deactivate()

In [27]:
model_lab3_Ex1.constraints[1].deactivate()

In [28]:
model_lab3_Ex1.constraints[4].deactivate()

In [29]:
new_constr_coef = [0,-1,-1]
new_constr_rhs = -25
model_lab3_Ex1.constraints.add(sum(new_constr_coef[j]*model_lab3_Ex1.x[j] for j in col_indices) <= new_constr_rhs)
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=5, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2]

In [30]:
obj_coef_c = np.array([1, 2, 3])
model_lab3_Ex1.new_objective = Objective(expr = summation(obj_coef_c,model_lab3_Ex1.x) )

In [31]:
model_lab3_Ex1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + 2*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints

In [32]:
result = opt_cbc.solve(model_lab3_Ex1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 79.5
  Upper bound: 79.5
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.015912532806396484
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


2(d). The solver status and solver termination condition messages obtained are:  
  
Solver status: ok  
Solver termination condition: optimal  
  


In [33]:
# display solution
print('\nObjective = ', model_lab3_Ex1.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab3_Ex1.x[i].value)

print('\nConstraints')
model_lab3_Ex1.constraints.display()


Objective =  79.5

Decision Variables
x[ 0 ] =  28.5
x[ 1 ] =  24.0
x[ 2 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body  : Upper
      2 :  None : -65.5 : -15.0
      3 :  None :  -8.0 :  -8.0
      5 :  None : -25.0 : -25.0


The value of the objective function = 79.5  
The values of decision variables are:  
x1: Denoted by x[0] =  28.5  
x2: Denoted by x[1] =  24.0  
x3: Denoted by x[2] =  1.0  
  
Constraints:  
The final constraints used in our problem are:  
  
x1 - 4x2 + 2x3 <= -15  
LHS = -65.5  
RHS = -15  
As LHS is not equal to the RHS, this constraint is inactive  
  
-2x1 + 2x2 + x3 <= -8.0  
LHS = -8  
RHS = -8  
As LHS is equal to the RHS, this constraint is active  
  
-x2 - x3 <= -25.0  
LHS = -25  
RHS = -25  
As LHS is equal to the RHS, this constraint is active